In [1]:
import pandas as pd
import re
import seaborn as sns
import os
import matplotlib.pyplot as plt
import requests
from lxml import html

In [2]:
def downloading_csv(path):
    """This function downloads from a raw link and saves the dataframe locally.
    args:
        :path: string.
    returns: 
        :df: pandas Dataframe
    """
    df=pd.read_csv(path, encoding='latin')
    return df

In [3]:
path1= 'data/happiness-2015.csv'
path2='data/happiness-2017.csv'
path3='data/happiness-2019.csv'
path4='data/happiness-2021.csv'

In [4]:
df= downloading_csv('data/happiness-2015.csv')
df.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [5]:
df2017= downloading_csv('data/happiness-2017.csv')
df.sample()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
114,Zimbabwe,Sub-Saharan Africa,115,4.61,0.0429,0.271,1.03276,0.33475,0.25861,0.08079,0.18987,2.44191


In [6]:
df2019= downloading_csv('data/happiness-2019.csv')
df.sample()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
73,Indonesia,Southeastern Asia,74,5.399,0.02596,0.82827,1.08708,0.63793,0.46611,0.0,0.51535,1.86399


In [7]:
df2021= downloading_csv('data/happiness-2021.csv')
df.sample()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
153,Rwanda,Sub-Saharan Africa,154,3.465,0.03464,0.22208,0.7737,0.42864,0.59201,0.55191,0.22628,0.67042


In [8]:
def downloading_html(path): 
    """ This function downloads the HTML content from a specified path, reads it into a pandas dataframe and returns the first table 
    in the HTML content.
    args:
        :path: string. the link
    returns: 
        :df: pandas Dataframe
    """
    
    headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    res = requests.get(path, headers=headers)
    table = pd.read_html(res.content, encoding = 'utf8')
    df = table[0]
    return df

In [9]:
df2=downloading_html('https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature')
df2.sample()

,Country,Average yearly temperature (1961–1990 Celsius)
22,Brunei,26.85


In [10]:
def cleaning_df2015(df, df2):
    """This function cleans and merges two dataframes based on a dictionary of country names and specific cleaning operations. 
    Args:
        :df: A pandas DataFrame with columns "Country", "Economy (GDP per Capita)",  and other columns that are not needed in 
        the final result.
        :df2: A pandas DataFrame with columns "Country", "Average yearly temperature (1961–1990 Celsius)".

    Returns:
        df: A merged DataFrame with the cleaned and renamed columns of both input dataframes.
    """
    
    countries_dict={'Denmark':'Denmark',
                    'Cyprus': 'Cyprus',
                    'Norway': 'Norway',
                    'Somalia': 'Somalia|Somaliland',
                    'Macedonia':'Macedonia',
                    'Swaziland': 'Eswatini',
                    'Democratic Republic of the Congo': 'Kinshasa', 
                    'Republic of the Congo':'Brazzaville'             
    }
    
    # Iterate over each element of the species2 column of the dataframe and replace the values for the keys
    for key, value in countries_dict.items():
        mask= df2['Country'].str.contains(value, case=False)
        df2.loc[mask, 'Country'] = key
    
    df2['Average yearly temperature (1961–1990 Celsius)'] = df2['Average yearly temperature (1961–1990 Celsius)'].str.replace('−', '-').astype(float)
    
    mask_Democratic = df['Country'].str.contains('Kinshasa', case=False)
    df.loc[mask_Democratic, 'Country'] = 'Democratic Republic of the Congo'
    
    mask_Congo = df['Country'].str.contains('Brazzaville', case=False)
    df.loc[mask_Congo, 'Country'] = 'Republic of the Congo'    
    
    mask_Somalia = df['Country'].str.contains('Somaliland', case=False)
    df.loc[mask_Somalia, 'Country'] = 'Somalia'
    
    df = pd.merge(df, df2, on='Country', how='outer')
    df = df.add_suffix(' 2015')
    df.rename(columns = {"Economy (GDP per Capita) 2015":"GDP per Capita 2015" ,
                         "Average yearly temperature (1961–1990 Celsius) 2015": "Average yearly temperature", 
                         "Country 2015": "Country", "Region 2015": "Region"}, inplace=True)
    
    df.drop(columns=["Region", "Standard Error 2015", "Family 2015", "Health (Life Expectancy) 2015", "Freedom 2015", 
                     "Trust (Government Corruption) 2015", "Generosity 2015", "Dystopia Residual 2015"], axis=1, inplace=True)
    
    
    return df
    

In [11]:
df = cleaning_df2015(df, df2)
df.sample()

,Country,Happiness Rank 2015,Happiness Score 2015,GDP per Capita 2015,Average yearly temperature
194,Liechtenstein,NaN,NaN,NaN,5.8


In [12]:
def cleaning_df2017(df):
    """This function cleans and renames columns of a dataframe of the 2017 happiness report and replaces some country names.
    
    Args:
        :df: A pandas DataFrame with columns "Country", "Happiness.Rank", "Happiness.Score", "Economy..GDP.per.Capita." and 
        other columns that are not needed in the final result.

    Returns:
        :df: A cleaned and renamed DataFrame with the specified columns for the 2017 happiness report. """
    
    df = df.add_suffix(' 2017')
    df.rename(columns = {"Country 2017" : "Country", 
                             "Happiness.Rank 2017" :"Happiness Rank 2017", 
                             "Happiness.Score 2017" : "Happiness Score 2017", 
                             "Economy..GDP.per.Capita. 2017": "GDP per Capita 2017"}, inplace=True)
    
    df.drop(columns=["Whisker.high 2017", "Whisker.low 2017", "Family 2017", "Health..Life.Expectancy. 2017", "Freedom 2017", 
                         "Generosity 2017", "Trust..Government.Corruption. 2017", "Dystopia.Residual 2017"], axis=1, inplace=True)
    
    mask_Taiwan = df['Country'].str.contains('Taiwan', case=False)
    df.loc[mask_Taiwan, 'Country'] = 'Taiwan'

    mask_HongKong = df['Country'].str.contains('Hong Kong', case=False)
    df.loc[mask_HongKong, 'Country'] = 'Hong Kong'

    mask_Democratic = df['Country'].str.contains('Kinshasa', case=False)
    df.loc[mask_Democratic, 'Country'] = 'Democratic Republic of the Congo'

    mask_Congo = df['Country'].str.contains('Brazzaville', case=False)
    df.loc[mask_Congo, 'Country'] = 'Republic of the Congo'

    return df
    

In [13]:
df2017=cleaning_df2017(df2017)
df2017.sample()

,Country,Happiness Rank 2017,Happiness Score 2017,GDP per Capita 2017
73,Jordan,74,5.336,0.991012


In [14]:
def cleaning_df2019(df):
    """This function cleans and renames columns of a dataframe of the 2019 happiness report and replaces some country names.
    
    Args:
        :df: A pandas DataFrame with columns "Country or region", "Overall rank", "Score", "GDP per Capita" and 
        other columns that are not needed in the final result.

    Returns:
        :df: A cleaned and renamed DataFrame with the specified columns for the 2019 happiness report. """
    
    
    df = df.add_suffix(' 2019')
    df.rename(columns = {"Country or region 2019" : "Country", 
                             "Overall rank 2019":"Happiness Rank 2019", 
                             "Score 2019": "Happiness Score 2019", 
                             "GDP per capita 2019":"GDP per Capita 2019" }, inplace=True)
    
    df.drop(columns=["Social support 2019", "Freedom to make life choices 2019", "Generosity 2019", 
                         "Perceptions of corruption 2019","Healthy life expectancy 2019"], axis=1, inplace=True)
    
    countries_dict={'Sudan':'Sudan',
                    'Cyprus': 'Cyprus',
                    'Somalia': 'Somalia|Somaliland',
                    'Macedonia':'Macedonia',
                    'Trinidad and Tobago': 'Trinidad & Tobago',
                    'Swaziland': 'Eswatini',
                    'Democratic Republic of the Congo': 'Kinshasa', 
                    'Republic of the Congo':'Brazzaville'     
    }
    
    # Iterate over each element of the species2 column of the dataframe and replace the values for the keys
    for key, value in countries_dict.items():
        mask= df['Country'].str.contains(value, case=False)
        df.loc[mask, 'Country'] = key
    
    return df

In [15]:
df2019=cleaning_df2019(df2019)
df2019.sample()

,Happiness Rank 2019,Country,Happiness Score 2019,GDP per Capita 2019
108,109,Cambodia,4.7,0.574


In [16]:
def cleaning_df2021(df):
    """This function cleans and renames columns of a dataframe of the 2021 happiness report and replaces some country names.
    It also creates a new column called "Happiness Rank 2021" with the rank of column "Ladder score".
    
    Args:
        :df: A pandas DataFrame with columns "ï»¿Country name", "Ladder score", "Explained by: Log GDP per capita 2021" and 
        other columns that are not needed in the final result.

    Returns:
        :df: A cleaned and renamed DataFrame with the specified columns for the 2021 happiness report. """
    
    df = df.add_suffix(' 2021')
    df.rename(columns = {"ï»¿Country name 2021" : "Country", 
                             "Ladder score 2021": "Happiness Score 2021", 
                             "Explained by: Log GDP per capita 2021":"GDP per Capita 2021"}, inplace=True)
    
    df.drop(columns=["Regional indicator 2021", "Standard error of ladder score 2021", "upperwhisker 2021", 
                         "lowerwhisker 2021","Social support 2021","Healthy life expectancy 2021", "Freedom to make life choices 2021", 
                         "Generosity 2021", "Perceptions of corruption 2021", "Ladder score in Dystopia 2021", 
                         "Explained by: Social support 2021", "Explained by: Healthy life expectancy 2021", 
                         "Explained by: Perceptions of corruption 2021", "Dystopia + residual 2021", "Explained by: Generosity 2021", 
                         "Explained by: Freedom to make life choices 2021","Logged GDP per capita 2021"  ], axis=1, inplace=True)
    
    mask_Taiwan = df['Country'].str.contains('Taiwan', case=False)
    df.loc[mask_Taiwan, 'Country'] = 'Taiwan'

    mask_Congo = df['Country'].str.contains('Brazzaville', case=False)
    df.loc[mask_Congo, 'Country'] = 'Republic of the Congo'

    mask_HongKong = df['Country'].str.contains('Hong Kong', case=False)
    df.loc[mask_HongKong, 'Country'] = 'Hong Kong'

    mask_Macedonia = df['Country'].str.contains('Macedonia', case=False)
    df.loc[mask_Macedonia, 'Country'] = 'Macedonia'
    
    df['Happiness Rank 2021'] = df['Happiness Score 2021'].rank(method='dense', ascending=False).astype(int)
    
    return df

In [17]:
df2021=cleaning_df2021(df2021)
df2021.sample()

,Country,Happiness Score 2021,GDP per Capita 2021,Happiness Rank 2021
53,Thailand,5.985,1.107,54


In [18]:
def merging_df(df,df1,df2,df3):
    """This function merges four pandas DataFrames by country name.
    Args:
        :df: A pandas DataFrame with columns "Country" and other columns to be merged.
        :df1, df2, df3: Other pandas DataFrames with "Country" column and other columns to be merged.

    Returns:
        :df: A merged DataFrame with all the columns from the four input DataFrames, merged by country name. 
    
    """
    merged_df = pd.merge(df, df1, on='Country', how='outer')
    merged_df2 = pd.merge(merged_df, df2, on='Country', how='outer')
    merged_df3 = pd.merge(merged_df2, df3, on='Country', how='outer')
    df=merged_df3
    
    return df

In [19]:
df=merging_df(df,df2017,df2019,df2021)
df.sample()

,Country,Happiness Rank 2015,Happiness Score 2015,GDP per Capita 2015,Average yearly temperature,Happiness Rank 2017,Happiness Score 2017,GDP per Capita 2017,Happiness Rank 2019,Happiness Score 2019,GDP per Capita 2019,Happiness Score 2021,GDP per Capita 2021,Happiness Rank 2021
100,Mongolia,100.0,4.874,0.82819,1.7,100.0,4.955,1.027236,83.0,5.285,0.948,5.677,0.966,69.0


In [20]:
def mean_columns(df):
    """ This functions addes a mean score of happiness and GDP per capita to a dataframe, eliminates rows with incomplete data 
    or irrelevant countries, and fills in missing values for temperature.

    Args:
        :df: a pandas Dataframe with happiness and GDP data

    Returns:
        :df: a pandas Dataframe with additional columns for mean happiness score and mean GDP per capita, as well as cleaned 
        and filled data."""
    
    df_score=df[['Country', 'Happiness Score 2015', 'Happiness Score 2017', 'Happiness Score 2019', 'Happiness Score 2021']]
    df_mean = df_score.loc[:, ['Happiness Score 2015', 'Happiness Score 2017', 'Happiness Score 2019', 'Happiness Score 2021']].mean(axis=1).round(2)
    df_score["Mean Score"]= df_mean
    df = df.dropna(subset=["Happiness Score 2015",
                       "Happiness Score 2017","Happiness Score 2019", 
                       "Happiness Score 2021"], how="all")
    
    eliminated_rows=['North Cyprus','Oman', 'Suriname', 'Belize', 'South Sudan', 'Maldives', 'Djibouti' ]
    for i in eliminated_rows:
        df = df.drop(df.loc[df['Country'] == i].index)
        
    df.loc[df['Country'] == 'Taiwan', 'Average yearly temperature'] = 27.0
    df.loc[df['Country'] == 'Kosovo', 'Average yearly temperature'] = 15.0
    df.loc[df['Country'] == 'Palestinian Territories', 'Average yearly temperature'] = 20
    
    df['Mean Happiness Score'] = df[['Happiness Score 2015', 'Happiness Score 2017', 'Happiness Score 2019', 
                                     'Happiness Score 2021']].mean(axis=1).round(3)
    
    df['Mean GDP per Capita'] = df[['GDP per Capita 2015', 'GDP per Capita 2017', 'GDP per Capita 2019', 
                                    'GDP per Capita 2021']].mean(axis=1).round(2)
    
    return df

In [21]:
df= mean_columns(df)
df.sample()

C:\Users\pagob\AppData\Local\Temp\ipykernel_19532\3582105330.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score["Mean Score"]= df_mean


,Country,Happiness Rank 2015,Happiness Score 2015,GDP per Capita 2015,Average yearly temperature,Happiness Rank 2017,Happiness Score 2017,GDP per Capita 2017,Happiness Rank 2019,Happiness Score 2019,GDP per Capita 2019,Happiness Score 2021,GDP per Capita 2021,Happiness Rank 2021,Mean Happiness Score,Mean GDP per Capita
135,Egypt,135.0,4.194,0.8818,22.1,104.0,4.735,0.989702,137.0,4.166,0.913,4.283,0.954,130.0,4.345,0.93


In [22]:
def population_df(df):
    """ This function retrieves the population data for a given list of countries from the World Bank API and merge it with a 
    dataframe of happiness scores. The merged dataframe will be sorted by the mean happiness score in descending order, and a 
    column for mean rank will be added.

    Args:
        :df: A pandas Dataframe of happiness scores for a list of countries.

    Returns:
        :df: A merged dataframe of happiness scores and population data, sorted by mean happiness score in descending order.
    """
    
    import wbdata

    # Define the indicators we want to retrieve
    indicators = {"SP.POP.TOTL": "Population"}

    # Define the countries we want to retrieve data for
    countries = ['FI', 'DK', 'CH', 'IS', 'NO', 'NL', 'SE', 'NZ', 'CA', 'AU', 'IL', 'AT', 'CR', 'LU', 'IE', 'US', 'DE', 'GB', 
             'BE', 'CZ', 'AE', 'MX', 'FR', 'BR', 'MT', 'SG', 'CL', 'QA', 'TW', 'PA', 'UY', 'SA', 'ES', 'GT', 'AR', 'CO', 
             'BH', 'TH', 'TT', 'SK', 'IT', 'KW', 'SV', 'UZ', 'SI', 'PL', 'LT', 'NI', 'EC', 'JP', 'KZ', 'KR', 'CY', 'XK', 
             'BO', 'JM', 'RO', 'EE', 'PE', 'MD', 'MU', 'LV', 'RU', 'PY', 'MY', 'HR', 'BY', 'RS', 'LY', 'PH', 'PT', 'HU', 
             'HK', 'HN', 'TM', 'VE', 'DZ', 'ME', 'BA', 'KG', 'ID', 'TR', 'GR', 'DO', 'PK', 'VN', 'CN', 'AZ', 'MN', 'TJ', 
             'MK', 'BT', 'NG', 'MA', 'JO', 'SO', 'LB', 'NP', 'LA', 'AL', 'BG', 'ZA', 'CM', 'GM', 'GH', 'MZ', 'BD', 'PS', 
             'IR', 'TN', 'AM', 'IQ', 'CG', 'NA', 'SN', 'KE', 'CI', 'UA', 'GE', 'GA', 'SZ', 'ZM', 'CD', 'MM', 'NE', 'KH', 
             'ET', 'MR', 'SL', 'LK', 'EG', 'ML', 'BF', 'BJ', 'UG', 'LR', 'IN', 'GN', 'TD', 'KM', 'LS', 'AO', 'MG', 'SD', 
             'HT', 'MW', 'ZW', 'BW', 'YE', 'TG', 'TZ', 'RW', 'BI', 'SY', 'AF', 'CF'
    ]

    # Retrieve the data from the API and store it in a dataframe
    df2= wbdata.get_dataframe(indicators, country=countries)
    df2= df2.loc[df2.index.get_level_values("date") == "2019"]
    df2.reset_index(inplace=True)
    df2=df2.drop(columns=['date'])
    df2 = df2.rename(columns={'country': 'Country'})
    
    countries_dict={'Czech Republic':'Czechia',
                'Slovakia':'Slovak Republic',
                'South Korea':'Korea, Rep',
                'Russia':'Russian Federation',
                'Hong Kong':'Hong Kong SAR, China',
                'Venezuela':'Venezuela, RB',
                'Kyrgyzstan':'Kyrgyz Republic',
                'Turkey':'Turkiye',
                'Macedonia':'North Macedonia',
                'Laos':'Lao PDR',
                'Gambia':'Gambia, The',
                'Iran':'Iran, Islamic Rep.',
                'Republic of the Congo':'Congo, Rep.',
                'Ivory Coast':"Cote d'Ivoire" ,
                'Swaziland':'Eswatini',
                'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
                'Egypt':'Egypt',
                'Yemen':'Yemen, Rep.',
                'Syria':'Syrian Arab Republic',
                'Palestinian Territories': 'West Bank and Gaza'
    }

    for key, value in countries_dict.items():
        mask= df2['Country'].str.contains(value, case=False)
        df2.loc[mask, 'Country'] = key
    
    merged_df = pd.merge(df, df2, on='Country', how='outer')
    merged_df.loc[merged_df['Country'] == 'Taiwan', 'Population'] = 23733876
    df=merged_df
    
    df['Mean Rank'] = df['Mean Happiness Score'].rank(method='dense', ascending=False).astype(int)
    
    df= df.sort_values("Mean Rank", ascending=True)
    df = df.reset_index(drop=True)
    
    return df

In [23]:
df=population_df(df)

In [33]:
import plotly.graph_objects as go
import plotly.offline as opy
import plotly.express as px
import plotly.graph_objects as go
import kaleido
import chart_studio
import plotly.io as pio

In [36]:
def happiness_temperature_scatter(df):
    """Creates a scatter plot using Plotly Express that shows the relationship between the mean happiness score and 
    the average yearly temperature for each country in the given dataframe.

    Args:
        :df: A pandas dataframe containing the data to be plotted. The dataframe has the columns named 'Mean Happiness Score', 
        'Average yearly temperature', and 'Country'.

    Returns:
        None. Displays the resulting plot in the output cell using Plotly.
    """
    
    fig = px.scatter(df, x="Mean Happiness Score", y="Average yearly temperature", 
                     color="Average yearly temperature", hover_name="Country", title="Countries happiness Score and their temperature")
    
    #fig.write_image('figures/happiness_temperature.png')
    pio.write_html(fig, file='figures/happiness_temperature_scatter.html', auto_open=True)
    fig.show()


In [37]:
happiness_temperature_scatter(df)

In [27]:
def happiness_temperature(df):
    """Creates a line plot using Plotly Express to visualize the relationship between the mean happiness score 
    and the average yearly temperature of different countries in the input DataFrame `df`.
    
    Args:
        :df: A pandas dataframe containing the following columns:
        - 'Country': the name of the country (str)
        - 'Mean Happiness Score': the mean happiness score of the country (float)
        - 'Average yearly temperature': the average yearly temperature of the country in Celsius (float)

    Return:
    None
        Displays the Plotly figure object containing the line plot of the mean happiness score and 
        the average yearly temperature of each country in the input DataFrame `df`. """ 

    fig = px.line(df, x="Mean Happiness Score", y="Average yearly temperature", hover_name="Country", color_discrete_sequence=['#00BFFF'])
    fig.update_layout(title='Countries happiness Score and their temperature', xaxis=dict(tickfont=dict(size=12)), yaxis=dict(tickfont=dict(size=12)))
    fig.update_traces(mode='markers+lines', marker=dict(size=8))
    
    fig.show()

In [28]:
happiness_temperature(df)

In [29]:
def happiness_worldmap(df):
    """Creates a choropleth map using Plotly to visualize the happiness rank and GDP per capita of different countries 
    in the input DataFrame `df`.

    Args:
        :df: A pandas dataframe containing the following columns:
        - 'Country': the name of the country (str)
        - 'Mean Rank': the mean happiness rank of the country (float)
        - 'Mean GDP per Capita': the mean GDP per capita of the country in US dollars (float)

    Return
    """
    import plotly.graph_objects as go
    import plotly.offline as opy

    fig = go.Figure(go.Choropleth(
        locations = df['Country'],
        locationmode = "country names",
        z = df['Mean Rank'],
        text = df['Mean GDP per Capita'],
        colorscale = 'bluyl',
        autocolorscale = False,
        reversescale = False,
        marker_line_color = '#efefef',
        marker_line_width = 0.5,
        colorbar_title = 'Happiness Rank',       
        )
    )
    fig.update_layout(
        title_text = 'Happiness Rank and GDP per capita',
        showlegend = False,
        geo = dict(
            scope = 'world',
            resolution = 50,
            projection_type = 'miller',
            showcoastlines = True,
            showocean = True,
            showcountries = True,
            oceancolor = '#eaeaea',
            lakecolor = '#eaeaea',
            coastlinecolor = '#dadada'
        )
    )
    fig.show()

In [30]:
happiness_worldmap(df)

In [31]:
def happiness_GDP(df):
    # Ordenar DataFrame según 'Mean GDP per Capita'
    df_sorted = df.sort_values(by='Mean GDP per Capita')

    # Crear gráfica con DataFrame ordenado
    fig = px.scatter(df_sorted, x='Mean Happiness Score', y='Mean GDP per Capita', size='Population', 
                 color='Average yearly temperature', hover_name='Country', log_x=True, size_max=60)

    fig.update_layout(title='Relation between Happiness, Population and GDP per Capita',
                  xaxis_title='Mean Happiness Score',
                  yaxis_title='Mean GDP per Capita',
                  legend_title='Country')
    fig.show()


In [32]:
happiness_GDP(df)